In [11]:
import pandas as pd

In [14]:
data = pd.read_csv("/home/lewagonvaleria/code/mfcorredor/DeepFeelings/raw_data/sentiment_tw_apple.csv")
data.head()

,date,text,sentiment
0,2022-06-01 10:51:36.379392,refund is now if i buy it will cost way more t...,Negative
1,2022-06-01 10:51:36.379392,especially if the is just be a more then be as...,Positive
2,2022-06-01 10:51:36.379392,best not to be too close to and can never be,Neutral
3,2022-06-01 10:51:36.379392,nee ich bin plus,Neutral
4,2022-06-01 10:51:36.379392,i swear to and it to have a space grey option too,Neutral


In [34]:
data["sentiment"] = data["sentiment"].apply(lambda x : x.capitalize())
data['date'] = pd.to_datetime(data['date'])
data.sort_values(by = ['date'] , inplace = True)
data['day'] = pd.to_datetime(data['date'].apply(lambda x : str(x).split(' ')[0]))
sentiment_data = data.groupby('day').agg( sentiment_fractions =
            ('sentiment',
            lambda x : x.value_counts().to_dict()))
polars = ['Negative','Neutral','Positive']

for sentiment in polars:
    sentiment_data.loc[:,sentiment] = sentiment_data['sentiment_fractions']\
                                        .apply(lambda sentiment_dict : sentiment_dict.get(sentiment))\
                                        .fillna(0.0)\
                                        .round(2)
sentiment_data.drop( columns = ['sentiment_fractions'] , inplace= True)
sentiment_data

,Negative,Neutral,Positive
day,,,
2022-06-01,515,825,257
2022-06-02,489,1390,362
2022-06-03,618,1606,388
2022-06-04,513,1386,310
2022-06-05,759,1281,166
2022-06-06,727,1695,256
2022-06-07,259,979,157


In [49]:
import plotly.graph_objects as go



fig = go.Figure()
fig.add_trace(go.Bar(
    x=sentiment_data.index,
    y=sentiment_data.loc[:,"Negative"],
    name='Negative Comments',
    marker_color='#E0427C'
))
fig.add_trace(go.Bar(
    x=sentiment_data.index,
    y=sentiment_data.loc[:,"Neutral"],
    name='Neutral Comments',
    marker_color='#F0CA47'
))

fig.add_trace(go.Bar(
    x=sentiment_data.index,
    y=sentiment_data.loc[:,"Positive"],
    name='Positive Comments',
    marker_color='#3CD3A9'
))

fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)
# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    xaxis_tickangle = - 45,
    title='Social Media Activity',
    xaxis = dict(title = "Date" , 
                  tickfont_size=14),
    yaxis=dict(
        title='Number of Comments',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)

fig.show()